In [1]:
import numpy as np
import pandas as pd

from talib import RSI, BBANDS, MACD, ATR, MOM, ROC

# Read in QQQ data

In [2]:
qqq = pd.read_csv('data/QQQ.csv', index_col=0)

qqq.drop('Close', axis=1, inplace=True)
qqq.columns = ['open', 'high', 'low', 'close', 'volume']
qqq.index.rename('date', inplace=True)

# Calculate technical indicators for QQQ

In [3]:
# Calculate RSI
qqq['rsi'] = RSI(qqq['close'], timeperiod=14)

# Calculate Bolinger Bands, take only high and low
high, mid, low = BBANDS(qqq['close'], timeperiod=20)
bbdf = pd.DataFrame({'bb_high': high, 'bb_low': low}, index=qqq.index)
qqq = qqq.join(bbdf)

# Take the log of the percentage difference between close price and bbands
qqq['bb_high'] = qqq.bb_high.sub(qqq.close).div(qqq.bb_high).apply(np.log1p)
qqq['bb_low'] = qqq.close.sub(qqq.bb_low).div(qqq.close).apply(np.log1p)

# Calculate the average true range: NOTE: he standardizes the result to make it more comparable across stocks
# but we're only focused one one stock so I didn't
qqq['atr'] = ATR(qqq.high, qqq.low, qqq.close, timeperiod=14)

# Calculate MACD: I think he standardizes here too, I won't again.
qqq['macd'] = MACD(qqq.close,fastperiod=12, slowperiod=26, signalperiod=9)[0]

# Calculate momentum
qqq['mom'] = MOM(qqq.close, timeperiod=10)

# Calculate rate-of-change
qqq['roc'] = ROC(qqq.close, timeperiod=10)

# Read in and prepare DXY

In [4]:
dxy = pd.read_csv('data/dxy.csv', index_col=0)
dxy.index = pd.to_datetime(dxy.index).rename('date')
dxy.sort_index(inplace=True)
dxy.columns = ['close', 'open', 'high', 'low', 'volume', 'dxy_daily']
dxy.drop('volume', axis=1, inplace=True)
dxy['dxy_daily'] = dxy.dxy_daily.str.replace('%', '').astype(float).div(100)

# Needed two csvs to cover the full date range due to the websites requirement.
dxy2 = pd.read_csv('data/dxy2.csv', index_col=0)
dxy2.index = pd.to_datetime(dxy2.index).rename('date')
dxy2.sort_index(inplace=True)
dxy2.columns = ['close', 'open', 'high', 'low', 'volume', 'dxy_daily']
dxy2.drop('volume', axis=1, inplace=True)
dxy2['dxy_daily'] = dxy2.dxy_daily.str.replace('%', '').astype(float).div(100)

# Combine and drop duplicate dates
dxy = pd.concat([dxy, dxy2])
dxy = dxy[~dxy.index.duplicated(keep='first')].sort_index()

# Create some rolling averages. NOTE: THESE ARE SIMPLE AVERAGES, not the geometric means used when calculating QQQ returns.
# We can change this if we want to but decided to keep it simple for now.
for w in [5,10,14,21,30]:
    dxy[f'dxy_rolling_{w}_day'] = dxy['dxy_daily'].rolling(window=w).mean()

dxy.drop(['close', 'open', 'high', 'low'], axis=1, inplace=True)

# Read in EEM

Not too sure how to process this, but will read in for now and compute some rolling windows

In [5]:
eem = pd.read_csv('data/EEM.csv', index_col=0)

eem.drop('Close', axis=1, inplace=True)
eem.columns = ['open', 'high', 'low', 'close', 'volume']
eem.index.rename('date', inplace=True)

# EMMA: Here would be where we did any sort of computations you need
eem['eem_daily'] = eem['close'].pct_change()
for w in [5,10,14,21,30]:
    eem[f'eem_rolling_{w}_day'] = eem['eem_daily'].rolling(window=w).mean()

eem.drop(['close', 'open', 'high', 'low', 'volume'], axis=1, inplace=True)

# Read in top QQQ stocks

In [7]:
# Note: Amazon seems to still be a part of the nasdaq, so included it here. Can drop that or TSLA if we wanna keep to 5

aapl = pd.read_csv('data/AAPL.csv', index_col=0)
amzn = pd.read_csv('data/AMZN.csv', index_col=0)
fb = pd.read_csv('data/FB.csv', index_col=0)
msft = pd.read_csv('data/MSFT.csv', index_col=0)
# goog = pd.read_csv('data/GOOG.csv', index_col=0)
tsla = pd.read_csv('data/TSLA.csv', index_col=0)

stocks = [aapl, amzn, fb, msft, tsla]
tickers = ['aapl', 'amzn', 'fb', 'msft', 'tsla']

for i in range(len(stocks)):
    stock = stocks[i]
    ticker = tickers[i]
    stock.drop('Close', axis=1, inplace=True)
    stock.columns = ['open', 'high', 'low', 'close', 'volume']
    stock.index.rename('date', inplace=True)
    
    # EMMA: Here would be where we did any sort of computations you need
    stock[f'{ticker}_daily'] = stock['close'].pct_change()
    
    for w in [5,10,14,21,30]:
        stock[f'{ticker}_rolling_{w}_day'] = stock[f'{ticker}_daily'].rolling(window=w).mean()

    stock.drop(['close', 'open', 'high', 'low', 'volume'], axis=1, inplace=True)

# Additional tickers

GLD and ^TNX. Following the same drill here, but just change the computation part for different metrics

In [8]:

gld = pd.read_csv('data/GLD.csv', index_col=0)
tnx = pd.read_csv('data/^TNX.csv', index_col=0)
vix = pd.read_csv('data/^VIX.csv', index_col=0)
jnk = pd.read_csv('data/JNK.csv', index_col=0)

others = [gld, tnx, vix, jnk]
tickers = ['gld', 'tnx', 'vix', 'jnk']

for i in range(len(others)):
    stock = others[i]
    ticker = tickers[i]
    stock.drop('Close', axis=1, inplace=True)
    stock.columns = ['open', 'high', 'low', 'close', 'volume']
    stock.index.rename('date', inplace=True)
    
    # EMMA: Here would be where we did any sort of computations you need
    stock[f'{ticker}_daily'] = stock['close'].pct_change()
    
    for w in [5,10,14,21,30]:
        stock[f'{ticker}_rolling_{w}_day'] = stock[f'{ticker}_daily'].rolling(window=w).mean()

    stock.drop(['close', 'open', 'high', 'low', 'volume'], axis=1, inplace=True)

# Putting it all together
Birth of the big-ass dataframe

In [9]:
for datasource in ([dxy, eem] + stocks + others):
    qqq = qqq.join(datasource)

# Let's add some QQQ returns

I'm gonna use the lags he used for this section, feel free to change them. Also gonna windsorize because that makes sense too

In [10]:
lags = [1, 5, 10, 21, 42, 63]
q = 0.0001

# These returns ARE the geometric means btw.
for lag in lags:
    qqq[f'return_{lag}d'] = (qqq.close
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower=x.quantile(q),
                                                       upper=x.quantile(1 - q)))
                                .add(1)
                                .pow(1 / lag)
                                .sub(1)
                                )

# Shifting the lags so we have multiple sets of returns for each date
for t in [1, 2, 3, 4, 5]:
    for lag in [1, 5, 10, 21]:
        qqq[f'return_{lag}d_lag{t}'] = (qqq[f'return_{lag}d'].shift(t * lag))

# Add some target variables

Simply shifting returns backwards so they can act as forward returns

In [11]:
for t in [1, 5, 10, 21]:
    qqq[f'target_{t}d'] = qqq[f'return_{t}d'].shift(-t)

# You wanted a big-ass dataframe?

...you got one

In [12]:
qqq

,open,high,low,close,volume,rsi,bb_high,bb_low,atr,macd,...,return_10d_lag4,return_21d_lag4,return_1d_lag5,return_5d_lag5,return_10d_lag5,return_21d_lag5,target_1d,target_5d,target_10d,target_21d
date,,,,,,,,,,,,,,,,,,,,,
1999-03-10,51.125000,51.156250,50.281250,44.523476,5232000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.004896,0.001951,-0.001169,0.004245
1999-03-11,51.437500,51.734375,50.312500,44.741455,9688600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.024361,0.004825,0.001932,0.003689
1999-03-12,51.125000,51.156250,49.656250,43.651520,8743600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.028714,0.004577,0.003321,0.004217
1999-03-15,50.437500,51.562500,49.906250,44.904942,6369000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.008495,-0.003544,0.004052,0.001199
1999-03-16,51.718750,52.156250,51.156250,45.286419,4905800,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.007220,-0.012334,0.002499,0.001440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-06,293.709991,295.359985,289.829987,294.609985,40959800,61.558118,0.018474,0.086460,7.118914,1.465873,...,-0.007314,0.003403,-0.025363,0.002017,0.006615,0.003905,-0.020434,NaN,NaN,NaN
2020-11-09,297.649994,299.140015,288.119995,288.589996,86537100,55.533787,0.035281,0.068016,7.397564,1.776441,...,-0.006103,0.004839,0.002227,0.002127,0.007244,0.002488,-0.017915,NaN,NaN,NaN
2020-11-10,285.170013,286.660004,280.619995,283.420013,69024900,50.924553,0.047727,0.051233,7.438453,1.587099,...,-0.005499,0.004798,0.017298,-0.000573,0.006887,0.003468,0.022370,NaN,NaN,NaN


In [16]:
qqq.to_csv('data/badf.csv')

In [18]:
pd.set_option('display.max_columns', None)
qqq

,open,high,low,close,volume,rsi,bb_high,bb_low,atr,macd,mom,roc,dxy_daily,dxy_rolling_5_day,dxy_rolling_10_day,dxy_rolling_14_day,dxy_rolling_21_day,dxy_rolling_30_day,eem_daily,eem_rolling_5_day,eem_rolling_10_day,eem_rolling_14_day,eem_rolling_21_day,eem_rolling_30_day,aapl_daily,aapl_rolling_5_day,aapl_rolling_10_day,aapl_rolling_14_day,aapl_rolling_21_day,aapl_rolling_30_day,amzn_daily,amzn_rolling_5_day,amzn_rolling_10_day,amzn_rolling_14_day,amzn_rolling_21_day,amzn_rolling_30_day,fb_daily,fb_rolling_5_day,fb_rolling_10_day,fb_rolling_14_day,fb_rolling_21_day,fb_rolling_30_day,msft_daily,msft_rolling_5_day,msft_rolling_10_day,msft_rolling_14_day,msft_rolling_21_day,msft_rolling_30_day,tsla_daily,tsla_rolling_5_day,tsla_rolling_10_day,tsla_rolling_14_day,tsla_rolling_21_day,tsla_rolling_30_day,gld_daily,gld_rolling_5_day,gld_rolling_10_day,gld_rolling_14_day,gld_rolling_21_day,gld_rolling_30_day,tnx_daily,tnx_rolling_5_day,tnx_rolling_10_day,tnx_rolling_14_day,tnx_rolling_21_day,tnx_rolling_30_day,vix_daily,vix_rolling_5_day,vix_rolling_10_day,vix_rolling_14_day,vix_rolling_21_day,vix_rolling_30_day,jnk_daily,jnk_rolling_5_day,jnk_rolling_10_day,jnk_rolling_14_day,jnk_rolling_21_day,jnk_rolling_30_day,return_1d,return_5d,return_10d,return_21d,return_42d,return_63d,return_1d_lag1,return_5d_lag1,return_10d_lag1,return_21d_lag1,return_1d_lag2,return_5d_lag2,return_10d_lag2,return_21d_lag2,return_1d_lag3,return_5d_lag3,return_10d_lag3,return_21d_lag3,return_1d_lag4,return_5d_lag4,return_10d_lag4,return_21d_lag4,return_1d_lag5,return_5d_lag5,return_10d_lag5,return_21d_lag5,target_1d,target_5d,target_10d,target_21d
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-03-10,51.125000,51.156250,50.281250,44.523476,5232000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0055,-0.00150,-0.00009,0.001236,0.001343,0.001147,NaN,NaN,NaN,NaN,NaN,NaN,-0.045788,-0.009339,-0.013481,-0.006731,-0.006582,-0.006788,0.055315,0.021704,0.022848,0.032631,0.013112,0.008057,NaN,NaN,NaN,NaN,NaN,NaN,-0.002704,0.015280,0.005547,0.007459,-0.000895,-0.001780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003287,-0.007936,-0.001522,0.002507,0.001339,0.002510,-0.009193,-0.029851,-0.008312,-0.013343,-0.008118,-0.004270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004896,0.001951,-0.001169,0.004245
1999-03-11,51.437500,51.734375,50.312500,44.741455,9688600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0056,-0.00378,-0.00011,0.000136,0.001233,0.000923,NaN,NaN,NaN,NaN,NaN,NaN,-0.011517,-0.007255,-0.013297,-0.009909,-0.006421,-0.006864,-0.016408,0.023888,0.008532,0.021582,0.016312,0.004460,NaN,NaN,NaN,NaN,NaN,NaN,0.000388,0.011849,0.005177,0.006506,0.000618,-0.001196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001542,-0.004481,-0.001368,0.001734,0.001412,0.002112,-0.016942,-0.016917,-0.012946,-0.011856,-0.010639,-0.005405,NaN,NaN,NaN,NaN,NaN,NaN,0.004896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.024361,0.004825,0.001932,0.003689
1999-03-12,51.125000,51.156250,49.656250,43.651520,8743600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0070,-0.00218,0.00048,0.000457,0.001210,0.001057,NaN,NaN,NaN,NaN,NaN,NaN,0.031069,0.000455,-0.004437,-0.010091,-0.006382,-0.006451,-0.011585,0.019281,0.004873,0.017512,0.016981,0.004803,NaN,NaN,NaN,NaN,NaN,NaN,-0.007743,0.006770,0.006602,0.005439,0.000082,-0.002517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008851,-0.006251,-0.004378,-0.000080,0.001217,0.001510,0.019286,0.006363,-0.010553,-0.002116,-0.008339,-0.002946,NaN,NaN,NaN,NaN,NaN,NaN,-0.024361,NaN,NaN,NaN,NaN,NaN,0.004896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028714,0.004577,0.003321,0.004217
1999-03-15,50.437500,51.562500,49.906250,44.904942,6369000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0026,-0.00160,-0.00067,0.000036,0.001267,0.000917,NaN,NaN,NaN,NaN,NaN,NaN,0.02636

In [19]:
import talib

# CANDLE STICK PATTERNS FOR QQQ

https://en.wikipedia.org/wiki/Candlestick_chart

In [34]:
with open('patterns.txt','r') as candlestick_patterns:
    patterns_list = candlestick_patterns.read()
    patterns_list = patterns_list.split()

In [36]:
qqq['CDL2CROWS']=talib.CDL2CROWS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDL3BLACKCROWS']=talib.CDL3BLACKCROWS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDL3INSIDE']=talib.CDL3INSIDE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDL3LINESTRIKE']=talib.CDL3LINESTRIKE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDL3OUTSIDE']=talib.CDL3OUTSIDE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDL3STARSINSOUTH']=talib.CDL3STARSINSOUTH(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDL3WHITESOLDIERS']=talib.CDL3WHITESOLDIERS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLABANDONEDBABY']=talib.CDLABANDONEDBABY(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLADVANCEBLOCK']=talib.CDLADVANCEBLOCK(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLBELTHOLD']=talib.CDLBELTHOLD(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLBREAKAWAY']=talib.CDLBREAKAWAY(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLCLOSINGMARUBOZU']=talib.CDLCLOSINGMARUBOZU(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLCONCEALBABYSWALL']=talib.CDLCONCEALBABYSWALL(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLCOUNTERATTACK']=talib.CDLCOUNTERATTACK(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLDARKCLOUDCOVER']=talib.CDLDARKCLOUDCOVER(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLDOJI']=talib.CDLDOJI(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLDOJISTAR']=talib.CDLDOJISTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLDRAGONFLYDOJI']=talib.CDLDRAGONFLYDOJI(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLENGULFING']=talib.CDLENGULFING(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLEVENINGDOJISTAR']=talib.CDLEVENINGDOJISTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLEVENINGSTAR']=talib.CDLEVENINGSTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLGAPSIDESIDEWHITE']=talib.CDLGAPSIDESIDEWHITE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLGRAVESTONEDOJI']=talib.CDLGRAVESTONEDOJI(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHAMMER']=talib.CDLHAMMER(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHANGINGMAN']=talib.CDLHANGINGMAN(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHARAMI']=talib.CDLHARAMI(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHARAMICROSS']=talib.CDLHARAMICROSS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHIGHWAVE']=talib.CDLHIGHWAVE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHIKKAKE']=talib.CDLHIKKAKE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHIKKAKEMOD']=talib.CDLHIKKAKEMOD(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLHOMINGPIGEON']=talib.CDLHOMINGPIGEON(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLIDENTICAL3CROWS']=talib.CDLIDENTICAL3CROWS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLINNECK']=talib.CDLINNECK(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLINVERTEDHAMMER']=talib.CDLINVERTEDHAMMER(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLKICKING']=talib.CDLKICKING(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLKICKINGBYLENGTH']=talib.CDLKICKINGBYLENGTH(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLLADDERBOTTOM']=talib.CDLLADDERBOTTOM(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLLONGLEGGEDDOJI']=talib.CDLLONGLEGGEDDOJI(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLLONGLINE']=talib.CDLLONGLINE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLMARUBOZU']=talib.CDLMARUBOZU(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLMATCHINGLOW']=talib.CDLMATCHINGLOW(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLMATHOLD']=talib.CDLMATHOLD(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLMORNINGDOJISTAR']=talib.CDLMORNINGDOJISTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLMORNINGSTAR']=talib.CDLMORNINGSTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLONNECK']=talib.CDLONNECK(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLPIERCING']=talib.CDLPIERCING(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLRICKSHAWMAN']=talib.CDLRICKSHAWMAN(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLRISEFALL3METHODS']=talib.CDLRISEFALL3METHODS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLSEPARATINGLINES']=talib.CDLSEPARATINGLINES(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLSHOOTINGSTAR']=talib.CDLSHOOTINGSTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLSHORTLINE']=talib.CDLSHORTLINE(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLSPINNINGTOP']=talib.CDLSPINNINGTOP(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLSTALLEDPATTERN']=talib.CDLSTALLEDPATTERN(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLSTICKSANDWICH']=talib.CDLSTICKSANDWICH(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLTAKURI']=talib.CDLTAKURI(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLTASUKIGAP']=talib.CDLTASUKIGAP(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLTHRUSTING']=talib.CDLTHRUSTING(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLTRISTAR']=talib.CDLTRISTAR(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLUNIQUE3RIVER']=talib.CDLUNIQUE3RIVER(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLUPSIDEGAP2CROWS']=talib.CDLUPSIDEGAP2CROWS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])
qqq['CDLXSIDEGAP3METHODS']=talib.CDLXSIDEGAP3METHODS(qqq['open'],qqq['high'],qqq['low'],qqq['close'])

In [37]:
qqq

,open,high,low,close,volume,rsi,bb_high,bb_low,atr,macd,mom,roc,dxy_daily,dxy_rolling_5_day,dxy_rolling_10_day,dxy_rolling_14_day,dxy_rolling_21_day,dxy_rolling_30_day,eem_daily,eem_rolling_5_day,eem_rolling_10_day,eem_rolling_14_day,eem_rolling_21_day,eem_rolling_30_day,aapl_daily,aapl_rolling_5_day,aapl_rolling_10_day,aapl_rolling_14_day,aapl_rolling_21_day,aapl_rolling_30_day,amzn_daily,amzn_rolling_5_day,amzn_rolling_10_day,amzn_rolling_14_day,amzn_rolling_21_day,amzn_rolling_30_day,fb_daily,fb_rolling_5_day,fb_rolling_10_day,fb_rolling_14_day,fb_rolling_21_day,fb_rolling_30_day,msft_daily,msft_rolling_5_day,msft_rolling_10_day,msft_rolling_14_day,msft_rolling_21_day,msft_rolling_30_day,tsla_daily,tsla_rolling_5_day,tsla_rolling_10_day,tsla_rolling_14_day,tsla_rolling_21_day,tsla_rolling_30_day,gld_daily,gld_rolling_5_day,gld_rolling_10_day,gld_rolling_14_day,gld_rolling_21_day,gld_rolling_30_day,tnx_daily,tnx_rolling_5_day,tnx_rolling_10_day,tnx_rolling_14_day,tnx_rolling_21_day,tnx_rolling_30_day,vix_daily,vix_rolling_5_day,vix_rolling_10_day,vix_rolling_14_day,vix_rolling_21_day,vix_rolling_30_day,jnk_daily,jnk_rolling_5_day,jnk_rolling_10_day,jnk_rolling_14_day,jnk_rolling_21_day,jnk_rolling_30_day,return_1d,return_5d,return_10d,return_21d,return_42d,return_63d,return_1d_lag1,return_5d_lag1,return_10d_lag1,return_21d_lag1,return_1d_lag2,return_5d_lag2,return_10d_lag2,return_21d_lag2,return_1d_lag3,return_5d_lag3,return_10d_lag3,return_21d_lag3,return_1d_lag4,return_5d_lag4,return_10d_lag4,return_21d_lag4,return_1d_lag5,return_5d_lag5,return_10d_lag5,return_21d_lag5,target_1d,target_5d,target_10d,target_21d,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-03-10,51.125000,51.156250,50.281250,44.523476,5232000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0055,-0.00150,-0.00009,0.001236,0.001343,0.001147,NaN,NaN,NaN,NaN,NaN,NaN,-0.045788,-0.009339,-0.013481,-0.006731,-0.006582,-0.006788,0.055315,0.021704,0.022848,0.032631,0.013112,0.008057,NaN,NaN,NaN,NaN,NaN,NaN,-0.002704,0.015280,0.005547,0.007459,-0.000895,-0.001780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003287,-0.007936,-0.001522,0.002507,0.001339,0.002510,-0.009193,-0.029851,-0.008312,-0.013343,-0.008118,-0.004270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004896,0.001951,-0.001169,0.004245,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1999-03-11,51.437500,51.734375,50.312500,44.741455,9688600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0056,-0.00378,-0.00011,0.000136,0.001233,0.000923,NaN,NaN,NaN,NaN,NaN,NaN,-0.011517,-0.007255,-0.013297,-0.009909,-0.006421,-0.006864,-0.016408,0.023888,0.008532,0.021582,0.016312,0.004460,NaN,NaN,NaN,NaN,NaN,NaN,0.000388,0.011849,0.005177,0.006506,0.000618,-0.001196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001542,-0.004481,-0.001368,0.001734,0.001412,0.002112,-0.016